In [1]:
import dice_ml

import pandas as pd

import torch.nn as nn
import torch

import copy

import os
import numpy as np
from pickle import dump , load
import seaborn as sns

In [2]:
class TaiwaneseCreditClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(23, 64)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(64, 128)
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(128, 32)
        self.act3 = nn.ReLU()
        self.hidden4 = nn.Linear(32, 16)
        self.act4 = nn.ReLU()
        self.output = nn.Linear(16, 1)
        self.act_output = nn.Sigmoid()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.dropout(self.act1(self.hidden1(x)))
        x = self.dropout(self.act2(self.hidden2(x)))
        x = self.dropout(self.act3(self.hidden3(x)))
        x = self.dropout(self.act4(self.hidden4(x)))
        # x = self.output(x)
        x = self.act_output(self.output(x))
        return x
    
    
def load_model():
    model = torch.load('./../model_training/Taiwanese_credit_model')
    model.eval()
    return model

def eval_model(model, input, scaler, columns_to_standardize):
    model.eval()
    # print(input)
    input = input.reshape(-1, 23)
    input = torch.tensor(input, dtype=torch.float32)
    input = input.numpy()
    input[:, columns_to_standardize] = scaler.transform(input[:, columns_to_standardize])
    input = torch.from_numpy(input).type(torch.float)
    with torch.no_grad():
        prob = model(input)
    return prob.tolist()[0][0]

def load_scaler(scaler_loc):
    return load(open(scaler_loc, 'rb'))

columns_to_standardize = list(range(23))

In [3]:
model = load_model()

In [4]:
def load_data(data):
    return np.load(data)


def create_dataframe_from_list_of_arrays(arrays, column_names=None):
    data= []
    if not arrays:
        raise ValueError("The list of arrays is empty")
    
    # Check if all arrays have the same length
    length = len(arrays[0])
    if not all(len(arr) == length for arr in arrays):
        raise ValueError("All arrays must have the same length")
    
    # Create a dictionary for DataFrame creation

    if len(column_names) != length:
        print(len(column_names), len(arrays))
        raise ValueError("Number of column names must match the number of arrays")
    for i in arrays:
        data.append({column_names[j]: i[j] for j in range(len(i))})
    return pd.DataFrame(data)


def create_data(folder_location, column_names):
    input_list = []
    for i in os.listdir(folder_location):
        if i=='.DS_Store':
            continue
        input_array_location = folder_location + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        print(input)
        input_list.append(input)
    
    df = create_dataframe_from_list_of_arrays(input_list, column_names)
    return df


In [7]:
def scale_input(scaler, input, columns_to_standardize):
    input = input.reshape(-1, 23)
    input = torch.tensor(input, dtype=torch.float32)
    input = input.numpy()
    input[:, columns_to_standardize] = scaler.transform(input[:, columns_to_standardize])
    return input


def calculate_o_1(phenotype, input, model, scaler, columns_to_standardize):
    mod_in = apply_phenotype(input, phenotype)
    o_1 = eval_model(model, mod_in, scaler, columns_to_standardize)
    return o_1

In [43]:
scaler = load_scaler('./../model_training/StandardScaler.pkl')
bounds= [(10000, 1000000),
              (0, 1),
              (0, 6),
              (0, 3),
              (20, 80),
              (-2, 8),
              (-2, 8),
              (-2, 8),
              (-2, 8),
              (-2, 8),
              (-2, 8),
              (-154973, 964511),
              (-67526, 983931),
              (-61506, 855086),
              (-81334, 891586),
              (53007, 927171),
              (-339603, 961664),
              (0, 873552),
              (0, 1215471),
              (0, 846040),
              (0, 621000),
              (0, 417990),
              (0, 403500)]
min_array = np.array([i[0] for i in bounds])
max_array = np.array([i[1] for i in bounds])
scaled_min = scale_input(scaler, min_array, columns_to_standardize)
scaled_max = scale_input(scaler, max_array, columns_to_standardize)

feature = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11','X12', 'X13',
            'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23']


feature_dict = {}

for i, j  in enumerate(bounds):
    feature_dict[feature[i]] = [scaled_min[0][i], scaled_max[0][i]]

feature_dict

{'X1': [-1.1320454, 6.6168604],
 'X2': [-0.83086455, 1.4070086],
 'X3': [-2.4233167, 5.3331966],
 'X4': [-2.944624, 2.7857614],
 'X5': [-1.6543822, 4.725081],
 'X6': [-1.7442763, 6.0858283],
 'X7': [-1.5366384, 5.869949],
 'X8': [-1.5043159, 5.9613605],
 'X9': [-1.4739374, 6.0869617],
 'X10': [-1.4650515, 6.2978134],
 'X11': [-1.4285109, 6.285492],
 'X12': [-2.7501092, 12.224778],
 'X13': [-1.6045824, 12.884915],
 'X14': [-1.5517751, 11.600624],
 'X15': [-1.8997005, 12.944077],
 'X16': [0.2083898, 14.328156],
 'X17': [-6.233661, 15.194207],
 'X18': [-0.34022838, 60.5825],
 'X19': [-0.27402094, 67.93915],
 'X20': [-0.2800343, 52.843742],
 'X21': [-0.3147646, 47.895416],
 'X22': [-0.29640576, 28.63767],
 'X23': [-0.28059798, 24.657183]}

In [44]:
print(scaled_max)

[[ 6.6168604  1.4070086  5.3331966  2.7857614  4.725081   6.0858283
   5.869949   5.9613605  6.0869617  6.2978134  6.285492  12.224778
  12.884915  11.600624  12.944077  14.328156  15.194207  60.5825
  67.93915   52.843742  47.895416  28.63767   24.657183 ]]


In [45]:
print(scaled_min)

[[-1.1320454  -0.83086455 -2.4233167  -2.944624   -1.6543822  -1.7442763
  -1.5366384  -1.5043159  -1.4739374  -1.4650515  -1.4285109  -2.7501092
  -1.6045824  -1.5517751  -1.8997005   0.2083898  -6.233661   -0.34022838
  -0.27402094 -0.2800343  -0.3147646  -0.29640576 -0.28059798]]


In [53]:
def apply_phenotype(array, phenotype):
    # Split the phenotype string into individual operations
    operations = phenotype.split(';')
    x = copy.deepcopy(array)
    # Iterate over each operation and execute it
    for operation in operations:
        # Strip any leading/trailing whitespace from the operation
        operation = operation.strip()
        
        # Use the exec function to execute the operation on the array
        if operation:
            exec(operation)

    return x


def inverse(dice_df_scaled, scaler, columns_to_standardize):
    df_values = dice_df_scaled.values
    print(df_values.shape)
    df_values[:, columns_to_standardize] = scaler.inverse_transform(df_values[:, columns_to_standardize])
    df_original = pd.DataFrame(df_values, columns=dice_df_scaled.columns)
    return df_original
    


def create_dice_cf(input_folder, scaler, model, columns_to_standardize):
    
    average_cf_count = []
    
    features = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11','X12', 'X13',
            'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23']
    backend = 'PYT'  # needs pytorch installed
    ML_modelpath = './../model_training/Taiwanese_credit_model'
    m = dice_ml.Model(model_path=ML_modelpath, backend=backend)
    
    d = dice_ml.Data(features={'X1': [-1.1320454, 6.6168604],
                                 'X2': [-0.83086455, 1.4070086],
                                 'X3': [-2.4233167, 5.3331966],
                                 'X4': [-2.944624, 2.7857614],
                                 'X5': [-1.6543822, 4.725081],
                                 'X6': [-1.7442763, 6.0858283],
                                 'X7': [-1.5366384, 5.869949],
                                 'X8': [-1.5043159, 5.9613605],
                                 'X9': [-1.4739374, 6.0869617],
                                 'X10': [-1.4650515, 6.2978134],
                                 'X11': [-1.4285109, 6.285492],
                                 'X12': [-2.7501092, 12.224778],
                                 'X13': [-1.6045824, 12.884915],
                                 'X14': [-1.5517751, 11.600624],
                                 'X15': [-1.8997005, 12.944077],
                                 'X16': [0.2083898, 14.328156],
                                 'X17': [-6.233661, 15.194207],
                                 'X18': [-0.34022838, 60.5825],
                                 'X19': [-0.27402094, 67.93915],
                                 'X20': [-0.2800343, 52.843742],
                                 'X21': [-0.3147646, 47.895416],
                                 'X22': [-0.29640576, 28.63767],
                                 'X23': [-0.28059798, 24.657183]}, 
                     outcome_name='outcome')

    exp = dice_ml.Dice(d, m)
    path_to_DICE_cf = 'Dice_cf'
    path_to_GE_cf = 'Ge_cf'
    
    for i in os.listdir(input_folder):
        if i=='.DS_Store':
            continue
        filename = input_folder + '/' + i + '/' "final_gen.csv"
        input_array_location = input_folder + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        print(input)
        input_scaled = scale_input(scaler, input, columns_to_standardize)
        # print(input_scaled[0])
        data = pd.read_csv(filename)
        
        data['o_1_mod'] = data.apply(lambda row: calculate_o_1(row['Phenotype'], input, model, scaler, columns_to_standardize), axis=1)
        data = data.sort_values(by=['o_1_mod'])
        data = data[data['o_1_mod'] > 0.50]
        
        number_of_counterfactuals = data.shape[0]
        average_cf_count.append(number_of_counterfactuals)
        input_dice = pd.DataFrame(data=[input_scaled[0]], columns=features)
        print(number_of_counterfactuals)
        # print(input_dice.head())
        # print(feature_dict)
        dice_exp = exp.generate_counterfactuals(input_dice, total_CFs=number_of_counterfactuals, desired_class="opposite")
        isExist = os.path.exists(path_to_DICE_cf)
        if not isExist:
            os.makedirs(path_to_DICE_cf)
        # print(dice_exp.cf_examples_list[0].final_cfs_df.head())
        dice_df_scaled = dice_exp.cf_examples_list[0].final_cfs_df
        dice_df_original = inverse(dice_df_scaled, scaler, columns_to_standardize)
        dice_df_original.to_csv(path_or_buf= path_to_DICE_cf + "/" + str(i) + '.csv'  , index=False)
        
        evolved_x = []
    
        for phenotype in data['Phenotype'].tolist():
            x_out = apply_phenotype(input, phenotype)
            evolved_x.append(x_out)
            
        GE_df = pd.DataFrame(data=evolved_x, columns=features)
        isExist = os.path.exists(path_to_GE_cf)
        if not isExist:
            os.makedirs(path_to_GE_cf)
        GE_df.to_csv(path_to_GE_cf + "/" + str(i) + '.csv'  , index=False)
        
    print(sum(average_cf_count)/len(average_cf_count))
        
    
        
    

In [54]:
input_folder = './../output/NSGAIII_multi/'
create_dice_cf(input_folder, scaler, model, columns_to_standardize)

[330000      1      1      1     34      0      0      0      0      0
      0 138009 141253 130567 128257 116988 105961   7015   7000  10007
   6500   6600   9000]
226


100%|██████████| 1/1 [00:00<00:00, 38.15it/s]


(226, 24)
[390000      0      1      1     38      0      0      0      0      0
      0 164418 167501 134282 128701 131529 135242   9000   7027   5000
   5000   6000   5000]
262


100%|██████████| 1/1 [00:00<00:00, 34.71it/s]

(262, 24)


...

100%|██████████| 1/1 [00:00<00:00, 36.11it/s]

(154, 24)
[180000      0      6      2     47      0      0      0      0      0
      0 167915 163279 166994 150812 123957  55778   6028   7758   5188
   4570   1876   1701]


154


100%|██████████| 1/1 [00:00<00:00, 36.42it/s]

(154, 24)
196.63333333333333


In [56]:
def create_NSGAII_cf(input_folder, model, scaler, columns_to_standardize):
    
    features =  ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11','X12', 'X13',
            'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23']

    

    path_to_NSGAII_cf = 'Ge_NSGAII_cf'
    
    for i in os.listdir(input_folder):
        if i=='.DS_Store':
            continue
        filename = input_folder + '/' + i + '/' "final_gen.csv"
        input_array_location = input_folder + '/' + i + '/' + 'input_data.npy'
        input = load_data(input_array_location)
        data = pd.read_csv(filename)

        
        # print(number_of_counterfactuals)
        
        data['o_1_mod'] = data.apply(lambda row: calculate_o_1(row['Phenotype'], input, model, scaler, columns_to_standardize), axis=1)
        data = data.sort_values(by=['o_1_mod'])
        data = data[data['o_1_mod'] > 0.50]
        number_of_counterfactuals = data.shape[0]
        
        evolved_x = []
    
        for phenotype in data['Phenotype'].tolist():
            x_out = apply_phenotype(input, phenotype)
            evolved_x.append(x_out)
            
        GE_df = pd.DataFrame(data=evolved_x, columns=features)
        isExist = os.path.exists(path_to_NSGAII_cf)
        if not isExist:
            os.makedirs(path_to_NSGAII_cf)
        GE_df.to_csv(path_to_NSGAII_cf + "/" + str(i) + '.csv'  , index=False)
        
        
input_folder = './../output/NSGAII_multi/'
create_NSGAII_cf(input_folder, model, scaler, columns_to_standardize)
        